## one example

In [ ]:
import matplotlib.pyplot as plt
from pycocotools.coco import COCO
import os
import skimage.io as io
import matplotlib.patches as patches
import numpy as np

In [ ]:
# load annotation file (gtsf predictions)
coco = COCO("/root/data/small_pen_data_collection/coco_body_parts_merged_27k.json")

In [ ]:
coco.cats

In [ ]:
print(len(coco.imgs.keys()))

In [ ]:
# create pairs list
pairs = {}
for (imgid, imgdata) in coco.imgs.items():
    # file_name = imgdata['coco_url'].split('%2F')[2].split('?alt')[0]
    # img_path = local_dic[file_name]
    img_path = imgdata['local_path']
    annotation_ids = coco.getAnnIds(imgIds=[imgid])
    if len(annotation_ids) == 0:
        continue
    if 'rectified' in img_path:
        ts = os.path.basename(img_path).split('.')[0].split('_')[-1]
        side = os.path.basename(img_path).split('.')[0].split('_')[0]
        if ts not in pairs:
            pairs[ts] = {}
        pairs[ts][side] = imgid

In [ ]:
print(len(pairs.keys()))

In [ ]:
# pick random timestamps
random_timestamp = np.random.choice(list(pairs.keys()))
# random_timestamp = '1539347092528'
# random_timestamp = '1539339593113'
random_timestamp = '1539775997086'

In [ ]:
# get image_data
left_image_data = coco.loadImgs([pairs[random_timestamp]['right']])[0]
right_image_data = coco.loadImgs([pairs[random_timestamp]['left']])[0]

# load images
left_image = io.imread(left_image_data['local_path'])
right_image = io.imread(right_image_data['local_path'])


# load annotations
left_annotations = coco.imgToAnns[left_image_data["id"]]
right_annotations = coco.imgToAnns[right_image_data["id"]]

# plot
f ,ax = plt.subplots(1, 2, figsize=(20, 20))

ax[0].imshow(left_image)
for ann in left_annotations:
    bbox = ann['bbox']
    rec = patches.Rectangle((bbox[1], bbox[0]), bbox[3]-bbox[1], bbox[2]-bbox[0], 
                            edgecolor='w', facecolor=None, fill=False, linestyle='--', linewidth=2)
    ax[0].add_patch(rec)
    seg = ann['segmentation'][0]
    poly = np.array(seg).reshape((int(len(seg)/2), 2))
    c = (np.random.random((1, 3))*0.6+0.4).tolist()[0]
    ax[0].add_patch(patches.Polygon(poly, facecolor=c, linewidth=0, alpha=0.4))
    ax[0].add_patch(patches.Polygon(poly, facecolor='none', edgecolor=c, linewidth=2))
    ax[0].axis('off')
    
ax[1].imshow(right_image)
for ann in right_annotations:
    bbox = ann['bbox']
    rec = patches.Rectangle((bbox[1], bbox[0]), bbox[3]-bbox[1], bbox[2]-bbox[0], 
                            edgecolor='w', facecolor=None, fill=False, linestyle='--', linewidth=2)
    ax[1].add_patch(rec)
    seg = ann['segmentation'][0]
    poly = np.array(seg).reshape((int(len(seg)/2), 2))
    polygon = patches.Polygon(poly)
    c = (np.random.random((1, 3))*0.6+0.4).tolist()[0]
    ax[1].add_patch(patches.Polygon(poly, facecolor=c, linewidth=0, alpha=0.4))
    ax[1].add_patch(patches.Polygon(poly, facecolor='none', edgecolor=c, linewidth=2))
    ax[1].axis('off')
plt.show()

### get world coordinates

In [ ]:
focal_length = 0.0107
baseline = 0.135
pixel_size_m = 3.45 * 1e-6 
focal_length_pixel = focal_length / pixel_size_m
image_sensor_width = 0.01412
image_sensor_height = 0.01412


def depth_from_disp(disp):
    depth = focal_length_pixel*baseline / np.array(disp)
    return depth


def convert_to_world_point(x, y, d):
    image_center_x = 3000 / 2.0 #depth_map.shape[1] / 2.0
    image_center_y = 4096 / 2.0# depth_map.shape[0] / 2.0
    px_x = x - image_center_x
    px_z = image_center_y - y

    sensor_x = px_x * (image_sensor_height / 3000)
    sensor_z = px_z * (image_sensor_width / 4096)
    
    # d = depth_map[y, x]
    world_y = d
    world_x = (world_y * sensor_x) / focal_length
    world_z = (world_y * sensor_z) / focal_length
    return np.array((world_x, world_y, world_z))

In [ ]:
coco.cats

In [ ]:
# centroids
world_coordinates = {}
for cat in range(2, 9):
    # get the seg
    left_seg = [ann for ann in left_annotations if ann['category_id'] == cat][0]['segmentation'][0]
    right_seg = [ann for ann in right_annotations if ann['category_id'] == cat][0]['segmentation'][0]
    
    # get the centroids
    poly = np.array(left_seg).reshape((int(len(left_seg)/2), 2))
    left_centroid = [np.mean(poly[:, 0]), np.mean(poly[:, 1])]
    
    poly = np.array(right_seg).reshape((int(len(right_seg)/2), 2))
    right_centroid = [np.mean(poly[:, 0]), np.mean(poly[:, 1])]
    
    # get the world coordinates
#     print(left_centroid, right_centroid)
    disparity = left_centroid[0] - right_centroid[0]
#     print(disparity)
    depth = depth_from_disp(disparity)
    world_coordinates[str(cat)] = convert_to_world_point(left_centroid[1], left_centroid[0], depth)

In [ ]:
category_ids = [str(ids) for ids in range(2, 9) ]
pairwise_distances = [c+k for (i, c) in enumerate(category_ids) for k in category_ids[i+1:]]

In [ ]:
dataset = {}
for d in pairwise_distances:
    dataset[d] = []

In [ ]:
for pair in pairwise_distances:
    if pair == 'ground_truth' or pair == 'left_image_key':
        continue
    cat0, cat1 = pair[0], pair[1]
    dist = np.linalg.norm(world_coordinates[cat0] - world_coordinates[cat1])
    dataset[pair].append(dist)

## relax the centroids

In [ ]:
import numpy as np
from tqdm import tqdm
import json
from statsmodels.iolib.smpickle import load_pickle
import pandas as pd

In [ ]:
thresh = 5
jitter = 20
nsimulation = 5000

In [ ]:
p = 0.0

In [ ]:
components = np.load("/root/data/models/biomass/components_sensitivity.npy")
iqr = json.load(open("/root/data/models/biomass/iqr_sensitivity.json"))

In [ ]:
model = load_pickle("/root/data/models/biomass/model_sensitivity.pickle")

In [ ]:
import copy

In [ ]:
weights = []
for i in tqdm(range(nsimulation)):
    
    world_coordinates = {}
    for cat in range(2, 9):
        # get the seg
        left_seg = [ann for ann in left_annotations if ann['category_id'] == cat][0]['segmentation'][0]
        right_seg = [ann for ann in right_annotations if ann['category_id'] == cat][0]['segmentation'][0]

        # get the centroids
        poly = np.array(left_seg).reshape((int(len(left_seg)/2), 2))
        left_centroid = [np.mean(poly[:, 0]), np.mean(poly[:, 1])] 
        coin = np.random.rand()
        if i > 0:
            if coin > p:
                left_centroid += np.random.uniform(low=-jitter, high=jitter, size=2)
        
        poly = np.array(right_seg).reshape((int(len(right_seg)/2), 2))
        right_centroid = [np.mean(poly[:, 0]), np.mean(poly[:, 1])]
        if i > 0:
            if coin > p:
                right_centroid += np.random.uniform(low=-jitter, high=jitter, size=2)
            
        # get the world coordinates
        disparity = left_centroid[0] - right_centroid[0]
        depth = depth_from_disp(disparity)
        world_coordinates[str(cat)] = convert_to_world_point(left_centroid[1], left_centroid[0], depth)
    
    dataset = {}
    for d in pairwise_distances:
        dataset[d] = []
    
    # stuff
    for pair in pairwise_distances:
        if pair == 'ground_truth' or pair == 'left_image_key':
            continue
        cat0, cat1 = pair[0], pair[1]
        dist = np.linalg.norm(world_coordinates[cat0] - world_coordinates[cat1])
        dataset[pair].append(dist)
    
    df = pd.DataFrame(dataset)
    for pwd in pairwise_distances:
        df[pwd] = df[pwd] / iqr[pwd]
    
    pidx = np.indices((df.shape[1], df.shape[1])).reshape(2, -1)
    lcol = pd.MultiIndex.from_product([df.columns, df.columns],  names=[df.columns.name, df.columns.name])
    X = pd.DataFrame(df.values[:, pidx[0]] * df.values[:, pidx[1]],  columns=lcol)
    
    # pick number of parts to jitter
    newX = np.dot(X, components.T)
    weight = np.squeeze(model.predict(newX))
    weights.append(weight)

In [ ]:
plt.hist(weights)
plt.plot([weights[0], weights[0]], [0, 2000], color="r")
plt.show()